In [5]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm

import os
import sys

sys.path.append("../../data_preprocess/")

import pickle
import re
from utils import json_pretty_dump, word2VecContinueLearning, trainWord2VecModelType2, tokenizeData, convertWord2Vec, text_cleansing, parse_datetime, parse_month
seed = 42
np.random.seed(seed)

# [mail.cup.com, mail.insect.com, mail.onion.com, mail.spiral.com] => choose one of this to ../processed_type2/AIT_preprocessed_type2/daemon/<Target>
data_dir = "../processed_type2/AIT_preprocessed_type2/daemon/mail.cup.com"

label_dir = [
    "mail.cup.com",
    "mail.insect.com",
    "mail.onion.com",
    "mail.spiral.com",
]

# [mail.cup.com, mail.insect.com, mail.onion.com, mail.spiral.com] => choose one of this to ../../Drain_result/AIT_daemon_<target>/daemon.log_structured.csv
# [mail.cup.com, mail.insect.com, mail.onion.com, mail.spiral.com] => choose one of this to ../../Drain_result/AIT_daemon_<target>/daemon.log_templates.csv
params = {
    "struct_file": "../../Drain_result/AIT_daemon_mail.cup.com/daemon.log_structured.csv",
    "template_file": "../../Drain_result/AIT_daemon_mail.cup.com/daemon.log_templates.csv",
    "label_file": f"../../AIT-LDS-v1_1/labels/{label_dir[0]}/apache2/mail.cup.com-access.log",
}

os.makedirs(data_dir, exist_ok=True)

In [6]:
struct_log = pd.read_csv(params['struct_file'])
template_log = pd.read_csv(params['template_file'])
labels = pd.read_csv(params['label_file'])
struct_log['time_label'] = labels['time_label']
struct_log['line_label'] = labels['line_label']

In [7]:
for index, row in struct_log.iterrows():
        struct_log.loc[index, "Month"] = parse_month(row["Month"])

In [8]:
struct_log.sort_values(["Month", "Day", "Time"], inplace=True)

In [9]:
split_date = struct_log[(struct_log['line_label'] != "0") & (struct_log['time_label'] != "0")]
split_date = split_date.iloc[0]

,LineId,Month,Day,Time,Type,Job,Content,EventId,EventTemplate,EventTemplateIdent,ParameterList,time_label,line_label


In [10]:
train_set = pd.DataFrame(columns=["LineId", "Month", "Day", "Time", "Type", "Job", "Content", "EventId", "EventTemplate", "EventTemplateIdent", "ParameterList", "time_label", "line_label"])
test_set = pd.DataFrame(columns=["LineId", "Month", "Day", "Time", "Type", "Job", "Content", "EventId", "EventTemplate", "EventTemplateIdent", "ParameterList", "time_label", "line_label"])

if len(split_date) > 0:
    for index , row in struct_log.iterrows():
        if (row.Month < split_date.Month):
            train_set = pd.concat([train_set, pd.DataFrame([row])], ignore_index=True)
        elif (row["Month"] == split_date["Month"]):
            if (row["Day"] < split_date["Day"]):
                train_set = pd.concat([train_set, pd.DataFrame([row])], ignore_index=True)
            elif (row["Day"] == split_date["Day"]):
                if row["Time"] < split_date["Time"]:
                    train_set = pd.concat([train_set, pd.DataFrame([row])], ignore_index=True)
                else:
                    test_set = pd.concat([test_set, pd.DataFrame([row])], ignore_index=True)
            else:
               test_set = pd.concat([test_set, pd.DataFrame([row])], ignore_index=True)
        else:
           test_set = pd.concat([test_set, pd.DataFrame([row])], ignore_index=True)

In [12]:
eventId_train = train_set.EventId.unique()
eventId_test = test_set.EventId.unique()

In [13]:
template_log_train = template_log[template_log["EventId"].isin(eventId_train)].reset_index()
template_log_test = template_log[template_log["EventId"].isin(eventId_test)].reset_index()
template_log_train["EventTemplateIdent_cleansed"] = template_log_train.EventTemplateIdent.map(text_cleansing)
template_log_test["EventTemplateIdent_cleansed"] = template_log_test.EventTemplateIdent.map(text_cleansing)

In [14]:
template_log_train_list = template_log_train["EventTemplateIdent_cleansed"].astype('str').tolist()
template_log_test_list = template_log_test["EventTemplateIdent_cleansed"].astype('str').tolist()

In [15]:
tokenizer = RegexpTokenizer(r'[A-Z][a-z]+|\w+')
token_train_list = [tokenizer.tokenize(sen) for sen in template_log_train_list]
token_test_list = [tokenizer.tokenize(sen) for sen in template_log_test_list]

template_log_train["EventTemplateIdent_token"] = pd.Series(token_train_list)
template_log_test["EventTemplateIdent_token"] = pd.Series(token_test_list)
map_token_train = { row[0]: row[1] \
    for row in tqdm(
        template_log_train[["EventId", "EventTemplateIdent_token"]].values,
        desc="Mapping ID & token"
        ) }
train_set["Token"] = train_set.EventId.map(lambda id: map_token_train[id])
map_token_test = { row[0]: row[1] \
    for row in tqdm(
        template_log_test[["EventId", "EventTemplateIdent_token"]].values,
        desc="Mapping ID & token"
        ) }
test_set["Token"] = test_set.EventId.map(lambda id: map_token_test[id])

/var/folders/jy/qhf5nsts0_77mx3gxd3731800000gn/T/ipykernel_35510/2881305911.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  template_log_train["EventTemplateIdent_token"] = pd.Series(token_train_list)


In [16]:
# [mail.cup.com, mail.insect.com, mail.onion.com, mail.spiral.com] => choose one of this to AIT_daemon_<Target>_word2Vec
trainWord2VecModelType2(token_train_list, "AIT_daemon_mail.cup.com_word2Vec")

start train word2Vec model. . . . .


In [ ]:
with open(os.path.join(data_dir, "train_set.pkl"), "wb") as fw:
    pickle.dump(train_set, fw)
with open(os.path.join(data_dir, "test_set.pkl"), "wb") as fw:
    pickle.dump(test_set, fw)
with open(os.path.join(data_dir, "template_train_set.pkl"), "wb") as fw:
    pickle.dump(template_log_train, fw)
with open(os.path.join(data_dir, "template_test_set.pkl"), "wb") as fw:
    pickle.dump(template_log_test, fw)